# 📊 Microsoft Fabric – Warehouse & Lakehouse Metrics

This notebook queries **Microsoft Fabric Warehouses and Lakehouses** to collect table-level metrics:

| Metric | Source |
|--------|--------|
| Warehouse / Lakehouse Name | Fabric REST API |
| Table Name | SQL Endpoint DMVs / REST API |
| Number of Rows | SQL Endpoint DMVs |
| Size on Disk | SQL Endpoint DMVs |
| Maintenance On/Off | Fabric REST API (table maintenance) |
| Maintenance Retention (Days) | Fabric REST API |

```mermaid
flowchart LR
    A["\ud83d\udd10 Fabric Auth"] --> B["\ud83d\udce1 Fabric REST API"]
    B --> C["\ud83d\udce6 List Warehouses"]
    B --> D["\ud83c\udfe0 List Lakehouses"]
    C --> E["\ud83d\udd17 SQL Endpoint"]
    D --> F["\ud83d\udd17 SQL Endpoint"]
    E --> G["\ud83d\udcca Table Metrics"]
    F --> G
    G --> H["\ud83d\udccb Consolidated Report"]
```

## Prerequisites
- This notebook runs **inside Microsoft Fabric** (not externally)
- Fabric workspace with Warehouse or Lakehouse items
- Pre-installed libraries: `sempy`, `mssparkutils`, `pandas`, `requests`, `pyspark`

## 0️⃣ Verify Fabric Environment

In [ ]:
# ── Verify Fabric Environment ─────────────────────────────────────────
# All libraries below are pre-installed in the Fabric PySpark runtime.

import sempy.fabric as fabric
from notebookutils import mssparkutils
import pandas as pd
import requests
import json
import re
from datetime import datetime

print("✅ Fabric environment verified")
print(f"   sempy version:      {fabric.__version__ if hasattr(fabric, '__version__') else 'OK'}")
print(f"   mssparkutils:       available")
print(f"   pandas version:     {pd.__version__}")
print(f"   spark:              {spark.version}")

## 1️⃣ Configuration

Set your Fabric workspace ID and optional filters below.  
You can find your **Workspace ID** in the Fabric portal URL:  
`https://app.fabric.microsoft.com/groups/<WORKSPACE_ID>/...`

In [ ]:
# ── Configuration ────────────────────────────────────────────────────

# ── Fabric Workspace ───────────────────────────────────────────────
# Set to None to use the current workspace, or specify a workspace GUID
TARGET_WORKSPACE_ID = None   # e.g. "a1b2c3d4-e5f6-7890-abcd-1234567890ef"

# ── Resolve workspace ID ────────────────────────────────────────────
if TARGET_WORKSPACE_ID:
    WORKSPACE_ID = TARGET_WORKSPACE_ID
else:
    WORKSPACE_ID = fabric.get_workspace_id()

# ── Fabric API ─────────────────────────────────────────────────────
FABRIC_API_BASE = "https://api.fabric.microsoft.com/v1"

# ── Authentication via mssparkutils ──────────────────────────────────
fabric_token = mssparkutils.credentials.getToken("pbi")
sql_token    = mssparkutils.credentials.getToken("https://database.windows.net")

HEADERS = {
    "Authorization": f"Bearer {fabric_token}",
    "Content-Type": "application/json"
}

# ── Display Options ────────────────────────────────────────────────
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.float_format', '{:,.2f}'.format)

# ── Validate – fetch workspace info ──────────────────────────────────
resp = requests.get(f"{FABRIC_API_BASE}/workspaces/{WORKSPACE_ID}", headers=HEADERS)
resp.raise_for_status()
ws = resp.json()
print(f"📊 Fabric Metrics Configuration")
print(f"   Workspace:    {ws['displayName']}")
print(f"   Workspace ID: {WORKSPACE_ID}")
print(f"   Capacity:     {ws.get('capacityId', 'N/A')}")
print(f"   API Base:     {FABRIC_API_BASE}")

## 2️⃣ Discover Warehouses & Lakehouses

Uses the Fabric REST API to list all Warehouses and Lakehouses in the workspace, including their SQL analytics endpoints.

In [ ]:
# ── Discover Warehouses & Lakehouses ────────────────────────────────

def list_fabric_items(workspace_id, item_type, headers):
    """List all items of a given type in a Fabric workspace (handles pagination)."""
    url = f"{FABRIC_API_BASE}/workspaces/{workspace_id}/{item_type}"
    items = []
    while url:
        resp = requests.get(url, headers=headers)
        resp.raise_for_status()
        data = resp.json()
        items.extend(data.get("value", []))
        url = data.get("continuationUri")
    return items

# ── Warehouses ─────────────────────────────────────────────────────
raw_warehouses = list_fabric_items(WORKSPACE_ID, "warehouses", HEADERS)
warehouses = []
for w in raw_warehouses:
    props = w.get("properties", {})
    conn_string = props.get("connectionString", "")
    sql_ep = conn_string.replace("jdbc:sqlserver://", "").split(";")[0] if conn_string else ""
    warehouses.append({
        "item_id":       w["id"],
        "item_name":     w["displayName"],
        "item_type":     "Warehouse",
        "sql_endpoint":  sql_ep,
        "database_name": w["displayName"],
    })
print(f"📦 Found {len(warehouses)} Warehouse(s)")

# ── Lakehouses ─────────────────────────────────────────────────────
raw_lakehouses = list_fabric_items(WORKSPACE_ID, "lakehouses", HEADERS)
lakehouses = []
for lh in raw_lakehouses:
    props = lh.get("properties", {})
    sql_ep = props.get("sqlEndpointProperties", {}).get("connectionString", "")
    lakehouses.append({
        "item_id":       lh["id"],
        "item_name":     lh["displayName"],
        "item_type":     "Lakehouse",
        "sql_endpoint":  sql_ep,
        "database_name": lh["displayName"],
    })
print(f"🏠 Found {len(lakehouses)} Lakehouse(s)")

# ── Combined list ───────────────────────────────────────────────────
all_items = warehouses + lakehouses
print(f"\n✅ Total: {len(all_items)} item(s) to scan")
for item in all_items:
    print(f"   {item['item_type']:10s} | {item['item_name']} | SQL: {item['sql_endpoint'][:60] if item['sql_endpoint'] else 'N/A'}")

## 3️⃣ Query Table Metrics via SQL Endpoint

Connects to each Warehouse/Lakehouse SQL analytics endpoint using **Spark JDBC** (always available in Fabric).  
Retrieves per-table: **row count**, **reserved size on disk**, **used size**.

In [ ]:
# ── Query Table Metrics via SQL Endpoint ─────────────────────
# Uses Spark's built-in JDBC connector (always available in Fabric)
# with the mssparkutils token for authentication.
#
# NOTE: Fabric SQL endpoints do NOT have sys.dm_pdw_nodes_db_partition_stats
# (that's a Synapse dedicated SQL pool DMV). We use:
#   - Primary:  sys.dm_db_partition_stats  (row counts + page sizes)
#   - Fallback: sys.partitions             (row counts only, no size)

# Primary query – uses sys.dm_db_partition_stats for rows + size
TABLE_METRICS_QUERY_PRIMARY = """(
SELECT
    s.name                          AS schema_name,
    t.name                          AS table_name,
    t.create_date                   AS created_date,
    t.modify_date                   AS modified_date,
    SUM(ps.row_count)               AS row_count,
    SUM(ps.reserved_page_count) * 8 AS reserved_size_kb,
    SUM(ps.used_page_count) * 8     AS used_size_kb
FROM sys.tables t
INNER JOIN sys.schemas s
    ON t.schema_id = s.schema_id
INNER JOIN sys.dm_db_partition_stats ps
    ON t.object_id = ps.object_id
WHERE ps.index_id IN (0, 1)
    AND t.is_ms_shipped = 0
GROUP BY s.name, t.name, t.create_date, t.modify_date
) AS tbl"""

# Fallback query – uses sys.partitions (always available, no size info)
TABLE_METRICS_QUERY_FALLBACK = """(
SELECT
    s.name              AS schema_name,
    t.name              AS table_name,
    t.create_date       AS created_date,
    t.modify_date       AS modified_date,
    SUM(p.rows)         AS row_count,
    CAST(-1 AS BIGINT)  AS reserved_size_kb,
    CAST(-1 AS BIGINT)  AS used_size_kb
FROM sys.tables t
INNER JOIN sys.schemas s
    ON t.schema_id = s.schema_id
INNER JOIN sys.partitions p
    ON t.object_id = p.object_id
WHERE p.index_id IN (0, 1)
    AND t.is_ms_shipped = 0
GROUP BY s.name, t.name, t.create_date, t.modify_date
) AS tbl"""


def query_sql_endpoint(sql_endpoint, database_name, access_token, query):
    """Query a Fabric SQL endpoint using Spark JDBC."""
    jdbc_url = (
        f"jdbc:sqlserver://{sql_endpoint};"
        f"database={database_name};"
        f"encrypt=true;"
        f"trustServerCertificate=false;"
        f"loginTimeout=30;"
    )
    spark_df = (
        spark.read.format("jdbc")
        .option("url", jdbc_url)
        .option("dbtable", query)
        .option("accessToken", access_token)
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")
        .load()
    )
    return spark_df.toPandas()


def query_with_fallback(sql_endpoint, database_name, access_token):
    """Try primary query (with size), fall back to partitions-only (no size)."""
    try:
        df = query_sql_endpoint(sql_endpoint, database_name, access_token, TABLE_METRICS_QUERY_PRIMARY)
        return df, "dm_db_partition_stats"
    except Exception as e1:
        print(f"      ℹ️  dm_db_partition_stats not available, trying sys.partitions ...")
        try:
            df = query_sql_endpoint(sql_endpoint, database_name, access_token, TABLE_METRICS_QUERY_FALLBACK)
            return df, "partitions"
        except Exception as e2:
            raise Exception(f"Both queries failed.\n  Primary: {e1}\n  Fallback: {e2}")


# ── Collect metrics from all items ────────────────────────────
all_metrics = []

for item in all_items:
    sql_ep    = item["sql_endpoint"]
    db_name   = item["database_name"]
    item_name = item["item_name"]
    item_type = item["item_type"]

    if not sql_ep:
        print(f"⚠️  Skipping {item_type} '{item_name}' – no SQL endpoint found")
        continue

    print(f"🔗 Connecting to {item_type}: {item_name} ...")
    try:
        df_tables, source = query_with_fallback(sql_ep, db_name, sql_token)
        df_tables["item_type"] = item_type
        df_tables["item_name"] = item_name
        df_tables["item_id"]   = item["item_id"]
        all_metrics.append(df_tables)

        print(f"   ✅ Retrieved {len(df_tables)} table(s)  [via {source}]")
    except Exception as e:
        print(f"   ❌ Error querying {item_name}: {e}")

if all_metrics:
    df_metrics = pd.concat(all_metrics, ignore_index=True)
    print(f"\n📊 Total: {len(df_metrics)} table(s) across {len(all_metrics)} item(s)")
else:
    df_metrics = pd.DataFrame()
    print("\n⚠️  No table metrics collected.")

## 4️⃣ Query Table Maintenance Settings

Fabric supports **automatic table maintenance** for Warehouses and Lakehouses:  
- **V-Order optimization** – reorganizes data for better compression & read performance  
- **File compaction** – merges small Parquet/Delta files  
- **Unreferenced file removal** – cleans orphaned files  

This cell queries the Fabric REST API for maintenance configuration per item.

In [ ]:
# ── Query Table Maintenance Settings ───────────────────────────────

def get_maintenance_settings(workspace_id, item_id, item_type):
    """
    Retrieve table maintenance settings for a Warehouse or Lakehouse.
    Returns dict with maintenance_enabled, retention_days, and details.
    """
    maintenance_info = {
        "maintenance_enabled": "Unknown",
        "retention_period_days": None,
        "vorder_enabled": "Unknown",
        "compaction_enabled": "Unknown",
    }

    try:
        if item_type == "Lakehouse":
            url = (f"{FABRIC_API_BASE}/workspaces/{workspace_id}"
                   f"/lakehouses/{item_id}")
            resp = requests.get(url, headers=HEADERS)
            resp.raise_for_status()
            data = resp.json()
            props = data.get("properties", {})

            maint_config = props.get("defaultTableMaintenanceConfiguration", {})
            if maint_config:
                maintenance_info["maintenance_enabled"] = "ON" if maint_config.get("enabled", False) else "OFF"

                vorder = maint_config.get("vOrderOptimization", {})
                maintenance_info["vorder_enabled"] = "ON" if vorder.get("enabled", False) else "OFF"

                retention = maint_config.get("unreferencedFileRetention", {})
                if retention.get("enabled", False):
                    period = retention.get("retentionPeriod", "P7D")
                    days = parse_iso_duration_days(period)
                    maintenance_info["retention_period_days"] = days

                compaction = maint_config.get("fileCompaction", {})
                maintenance_info["compaction_enabled"] = "ON" if compaction.get("enabled", False) else "OFF"
            else:
                maintenance_info["maintenance_enabled"] = "OFF"

        elif item_type == "Warehouse":
            url = (f"{FABRIC_API_BASE}/workspaces/{workspace_id}"
                   f"/warehouses/{item_id}")
            resp = requests.get(url, headers=HEADERS)
            resp.raise_for_status()

            # Warehouse auto-maintenance is always on (built-in)
            maintenance_info["maintenance_enabled"] = "ON (Auto)"
            maintenance_info["vorder_enabled"]      = "ON (Auto)"
            maintenance_info["compaction_enabled"]   = "ON (Auto)"
            maintenance_info["retention_period_days"] = None  # Managed by Fabric

    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:
            maintenance_info["maintenance_enabled"] = "Not Available"
        else:
            maintenance_info["maintenance_enabled"] = f"Error: {e.response.status_code}"
    except Exception as e:
        maintenance_info["maintenance_enabled"] = f"Error: {str(e)[:50]}"

    return maintenance_info


def parse_iso_duration_days(duration_str):
    """Parse ISO 8601 duration string (e.g. 'P7D', 'P30D') to days."""
    if not duration_str:
        return None
    match = re.match(r"P(?:(\d+)D)?", duration_str)
    if match and match.group(1):
        return int(match.group(1))
    return None


# ── Collect maintenance settings for all items ──────────────────────
maintenance_data = []

for item in all_items:
    print(f"🔍 Checking maintenance for {item['item_type']}: {item['item_name']} ...")
    settings = get_maintenance_settings(WORKSPACE_ID, item["item_id"], item["item_type"])
    maintenance_data.append({
        "item_id":               item["item_id"],
        "item_name":             item["item_name"],
        "item_type":             item["item_type"],
        "maintenance_enabled":   settings["maintenance_enabled"],
        "retention_period_days": settings["retention_period_days"],
        "vorder_enabled":        settings["vorder_enabled"],
        "compaction_enabled":    settings["compaction_enabled"],
    })
    print(f"   Maintenance: {settings['maintenance_enabled']}")
    if settings["retention_period_days"]:
        print(f"   Retention:   {settings['retention_period_days']} days")

df_maintenance = pd.DataFrame(maintenance_data)

print(f"\n✅ Maintenance settings collected for {len(maintenance_data)} item(s)")
if not df_maintenance.empty:
    print()
    print(df_maintenance.to_string(index=False))

## 5️⃣ Consolidated Metrics Report

Merges table-level metrics with item-level maintenance settings into a single report.

In [ ]:
# ── Build Consolidated Report ──────────────────────────────────────

def format_size(size_kb):
    """Convert KB to human-readable size string."""
    if pd.isna(size_kb) or size_kb == 0:
        return "0 KB"
    if size_kb < 1024:
        return f"{size_kb:,.0f} KB"
    elif size_kb < 1024 * 1024:
        return f"{size_kb / 1024:,.2f} MB"
    else:
        return f"{size_kb / (1024 * 1024):,.2f} GB"


if not df_metrics.empty and not df_maintenance.empty:
    # Merge table metrics with maintenance settings
    df_report = df_metrics.merge(
        df_maintenance[["item_id", "maintenance_enabled", "retention_period_days",
                        "vorder_enabled", "compaction_enabled"]],
        on="item_id",
        how="left"
    )

    # Format size columns for display
    df_report["reserved_size_display"] = df_report["reserved_size_kb"].apply(format_size)
    df_report["used_size_display"]     = df_report["used_size_kb"].apply(format_size)

    # Format retention days
    df_report["retention_days_display"] = df_report["retention_period_days"].apply(
        lambda x: f"{int(x)} days" if pd.notna(x) else "N/A (Auto-managed)"
    )

    # Select and rename columns for display
    df_display = df_report[[
        "item_type", "item_name", "schema_name", "table_name",
        "row_count", "reserved_size_display", "used_size_display",
        "maintenance_enabled", "retention_days_display",
        "vorder_enabled", "compaction_enabled"
    ]].rename(columns={
        "item_type":              "Type",
        "item_name":              "Warehouse / Lakehouse",
        "schema_name":            "Schema",
        "table_name":             "Table Name",
        "row_count":              "Row Count",
        "reserved_size_display":  "Size on Disk",
        "used_size_display":      "Used Size",
        "maintenance_enabled":    "Maintenance",
        "retention_days_display": "Retention",
        "vorder_enabled":         "V-Order",
        "compaction_enabled":     "Compaction",
    })

    print("📋 FABRIC WAREHOUSE & LAKEHOUSE METRICS REPORT")
    print("=" * 120)
    print(f"   Workspace: {ws['displayName']}")
    print(f"   Items:     {len(all_items)} ({len(warehouses)} Warehouse, {len(lakehouses)} Lakehouse)")
    print(f"   Tables:    {len(df_display)}")
    total_rows = df_report['row_count'].sum()
    total_size = df_report['reserved_size_kb'].sum()
    print(f"   Total Rows: {total_rows:,.0f}")
    print(f"   Total Size: {format_size(total_size)}")
    print("=" * 120)
    print()
    print(df_display.to_string(index=False))
else:
    print("⚠️  No data to display. Ensure previous cells ran successfully.")

## 6️⃣ Summary by Warehouse / Lakehouse

Aggregated view showing totals per item with maintenance status.

In [ ]:
# ── Summary by Item ────────────────────────────────────────────────

if not df_metrics.empty and not df_maintenance.empty:
    df_summary = df_report.groupby(["item_type", "item_name", "maintenance_enabled",
                                     "retention_days_display"]).agg(
        table_count   = ("table_name", "count"),
        total_rows    = ("row_count", "sum"),
        total_size_kb = ("reserved_size_kb", "sum"),
    ).reset_index()

    df_summary["total_size"] = df_summary["total_size_kb"].apply(format_size)

    df_summary_display = df_summary[[
        "item_type", "item_name", "table_count", "total_rows",
        "total_size", "maintenance_enabled", "retention_days_display"
    ]].rename(columns={
        "item_type":              "Type",
        "item_name":              "Name",
        "table_count":            "Tables",
        "total_rows":             "Total Rows",
        "total_size":             "Total Size",
        "maintenance_enabled":    "Maintenance",
        "retention_days_display": "Retention",
    })

    print("📊 SUMMARY BY WAREHOUSE / LAKEHOUSE")
    print("=" * 100)
    print()
    print(df_summary_display.to_string(index=False))

    # ── Maintenance Status Summary ────────────────────────────────────
    print("\n")
    print("🔧 MAINTENANCE STATUS")
    print("-" * 60)
    for _, row in df_summary.iterrows():
        icon = "✅" if "ON" in str(row["maintenance_enabled"]) else "❌"
        retention = row["retention_days_display"]
        print(f"   {icon} {row['item_name']} ({row['item_type']})")
        print(f"      Maintenance: {row['maintenance_enabled']}")
        print(f"      Retention:   {retention}")
        print(f"      Tables: {row['table_count']:,}  |  Rows: {row['total_rows']:,.0f}  |  Size: {row['total_size']}")
        print()
else:
    print("⚠️  No data to summarize.")

## 7️⃣ Export Report to CSV

Saves the full metrics report to a CSV file for further analysis or sharing.

In [ ]:
# ── Export to CSV ───────────────────────────────────────────────────

if not df_metrics.empty:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"fabric_metrics_{timestamp}.csv"

    export_df = df_report[[
        "item_type", "item_name", "schema_name", "table_name",
        "row_count", "reserved_size_kb", "used_size_kb",
        "created_date", "modified_date",
        "maintenance_enabled", "retention_period_days",
        "vorder_enabled", "compaction_enabled"
    ]].copy()

    export_df.to_csv(output_file, index=False)
    print(f"✅ Report exported to: {output_file}")
    print(f"   Rows: {len(export_df)}")
    print(f"   Columns: {', '.join(export_df.columns)}")
else:
    print("⚠️  No data to export.")

---

## 📝 Notes

### Running This Notebook
- **Import** into any Fabric workspace via the deployment notebook or Fabric portal
- **Attach a Lakehouse** (optional) to enable CSV export and Delta table persistence
- Set `TARGET_WORKSPACE_ID` in Cell 5 to scan a different workspace, or leave `None` for the current one

### Table Maintenance in Microsoft Fabric

| Feature | Warehouse | Lakehouse |
|---------|-----------|----------|
| **Auto-Statistics** | Always ON (built-in) | N/A (Delta-based) |
| **V-Order Optimization** | Always ON (auto) | Configurable (ON/OFF) |
| **File Compaction** | Always ON (auto) | Configurable (ON/OFF) |
| **Unreferenced File Removal** | Auto-managed | Configurable with retention days |
| **Maintenance Schedule** | Automatic | User-configurable |

### Maintenance Retention Days (Lakehouse)
- Default: **7 days** for unreferenced file retention
- Configurable from **1 to 90 days** via Fabric portal or REST API
- Lower values → faster cleanup, less time-travel capability
- Higher values → more storage cost, longer rollback window

### SQL Endpoint DMVs Used
- `sys.tables` – table metadata
- `sys.schemas` – schema names
- `sys.dm_db_partition_stats` – row counts and page-level size metrics

### Fabric-Native Libraries Used
- `sempy.fabric` – workspace context and item discovery
- `mssparkutils.credentials` – automatic Entra ID token acquisition
- `spark.read.format("jdbc")` – Spark JDBC connector for SQL endpoint queries
- `notebookutils.mssparkutils` – file system operations